# 5- Experimento y Evaluación de Desempeño:

## a) Búsqueda en texto:
Evalúe el rendimiento de su implementación del índice invertido comparándolo con los
resultados obtenidos utilizando PostgreSQL. Para ello:
- Diseñe consultas equivalentes en ambos sistemas y mida el tiempo de respuesta y la calidad
de los resultados (¿Se obtiene los mismo resultados?).
- Investigue y explique cómo PostgreSQL realiza la recuperación de información textual:
    - El tipo de índice que utiliza (por ejemplo, GIN o GiST).
    - La función de similitud o ranking que emplea (ts_rank, ts_rank_cd, entre otras).
    - Cómo gestiona el procesamiento de consultas en campos tsvector y tsquery.


In [ ]:
import os
import psycopg2
from psycopg2.extras import execute_values
import pandas as pd
import time
from database import create_table, insert_record, check_table_exists, drop_table
from storage.Record import Record

csv_path = os.path.join(os.getcwd(), "news_csv")
table_name = "news_text"

def load_to_our_db(csv_path: str, table_name: str,) -> float:
    df = pd.read_csv(csv_path)
    schema = [
        ("id", "i"),
        ("title", "text"),
        ("content", "text"),
        ("year", "i"),
        ("author", "30s")
    ]
    if check_table_exists(table_name):
        drop_table(table_name)
    create_table(table_name, schema)
    start = time.time()
    for _, row in df.iterrows():
        values = [
            int(row["id"]),
            str(row["title"]),
            str(row["content"]),
            int(row["year"]),
            str(row["author"] or "")
        ]
        rec = Record(schema, values)
        insert_record(table_name, rec)
    end = time.time()
    return end - start

def load_to_postgres(csv_path: str, table_name: str, sc) -> float:
    connect = psycopg2.connect(dbname="postgres", user="postgres", password="postgres", host="localhost", port="5432")
    postgres = connect.cursor()
    postgres.autocommit = True
    postgres.execute(f"drop table if exists {table_name}")
    postgres.execute(f"""
        create table if not exists {table_name} (
            id integer primary key,
            title text,
            content text,
            year integer,
            author varchar(30)
        )
    """)
    start = time.time()
    with open(csv_path, 'r', encoding="utf-8") as f:
        postgres.copy_expert(f"copy {table_name} (id, title, content, year, author) from stdin with (format csv, header)", f)
    end = time.time()
    postgres.commit()
    postgres.close()
    connect.close()
    return end - start

